In [9]:
import gspread


'''Подключение таблицы
    в client_secrets.json содержится данные для подключения к проекту с тестовым заданием Google cloud
'''
gs = gspread.service_account(filename='service_account.json') # Подключение к сервису
sh_orig = gs.open_by_key('1syKH39TurHv9AVuUGcS-o9r51xw5nT2yQ6eEV3XM-PQ')# Инициализация таблицы
worksheet_orig = sh_orig.sheet1 #Указание листа
orig_data = worksheet_orig.get_all_values() #Получение данных


ModuleNotFoundError: No module named 'gspread'

In [2]:
import pandas as pd


df_orig = pd.DataFrame(orig_data[1:], columns=orig_data[0]) #Перевод данных в формат DataFrame


In [3]:
required_col = ['area', 'cluster', 'cluster_name', 'keyword', 'x', 'y', 'count'] # Требуемые колонки
for col in df_orig.columns: #Цикл по колонкам считаемого DataFrame df_orig
    if col not in required_col: #Условие удаления колонки
        del df_orig[col] #Удаление колонки


In [4]:
#Удаление дубликатов по условию, что в одной 'area' не может быть 2 одинаковых 'keyword'
df_clean = df_orig.drop_duplicates(subset=['keyword', 'area',])


In [5]:
#Ожидаемый тип данных
expect_type = {
    'area': str,
    'cluster': int,
    'cluster_name': str,
    'keyword': str,
    'count': int,
    'x': float,
    'y': float
}


In [ ]:
from view_clear import convert_to_expected_type


for col, dtype in expect_type.items(): # Прохолим циклом по ключам и значением словаря 
    df_clean[col] = df_clean[col].apply( # Меняем тип колонки
        convert_to_expected_type, # Вызываем фукцию которая меняет тип
        expected_type=dtype # Ожидаемый тип 
    )


In [7]:
for col, dtype in expect_type.items(): # Прохолим циклом по ключам и значением словаря 
    mask = df_clean[col].apply(lambda x: isinstance(x, dtype)) # Создание логического массива используя функцию isinstance по каждому элементу массива, тем самым помечая данные нежелательно формата
    df_clean = df_clean[mask] # наполнение данным только данными с маской == True


In [8]:
# Сортируем очищенные данные 
df_sort = df_clean.sort_values(
    by=['area', 'cluster', 'count'], # Сортируемые столбцы
    ascending=[True, True, False] # Логика сортировки True-по возрастающему, False-по убывающему)
)


In [9]:
palettes = [ # Палитры цветов из 5 наборов (на выбор)
    ['#39737c', '#98d9e4', '#b85a0d', '#32a251'],
    ['#ffd94a', '#3cb7cc', '#82853b', '#ff7f0f'],
    ['#86b4a9', '#acb877', '#b85a0d', '#32a251'],
    ['#ccc94d', '#39737c', '#86b4a9', '#ff7f0f'],
    ['#98d9e4', '#3cb7cc', '#ffb977', '#32a251']
]


In [10]:
cluster_colors = dict(zip(df_sort['cluster_name'].unique(), palettes[1])) #Формируем словарь соответвующий значению уникального кластера и цвета 2-ого набора палитры


In [11]:
df_sort['color'] = df_sort['cluster_name'].map(cluster_colors) # Присваваем каждой строке уникального кластера цвет

In [ ]:
sh_clean = gs.open_by_key('1yuCSKE7hbbX_ChOu9vCFJVdnAkC3T9rSg9pFpZIulOA') # Идификатор таблицы записи результата
worksheet_clean = sh_clean.sheet1 # Указание листа
# Записываем данные в Google Spreadsheet
worksheet_clean.clear() # Очищаем таблицу перед записью
data_list = [df_sort.columns.tolist()] + df_sort.values.tolist() # Превращаем в списоки DataFrame(требование функции insert_rows) 
worksheet_clean.insert_rows(data_list) #вставляем данные в таблицу
end_row = len(data_list) # Количество строк в списке
end_col = len(data_list[0]) # Количество столбцов
worksheet_clean.set_basic_filter(1, 1, end_row, end_col)  # Устанавливаем фильтры
worksheet_clean.freeze(rows=1) # Закрепляем заголовки


In [7]:
2+2

4